<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [20]:
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
import joblib
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [21]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "bootless"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-bootless-16072025


## Step 1.2: Configure STAC access and search parameters

In [4]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [5]:
# ## Use training data bounds

training = gpd.read_file("training-data/nm-bootless-bay-13072025_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

# TODO: configure colours...
# training.explore(column="observed")

In [6]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [7]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 5MB
Dimensions:      (y: 490, x: 262, time: 1)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/16)
    nir          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    red          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    blue         (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    green        (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    emad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    smad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 257kB dask.arr

In [8]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

/srv/conda/envs/notebook/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36697 instead
  warnings.warn(


In [9]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [10]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

In [11]:
land_masked_ds = mask_land(scaled, return_mask = True)

In [12]:
deeps_masked_ds, deeps_mask = mask_deeps(scaled, return_mask = True)

In [13]:
elevation_masked_ds, elevation_mask = mask_elevation(scaled, return_mask = True)

In [14]:
masked_scaled, mask = all_masks(scaled, return_mask = True)
# mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [15]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = masked_scaled.blue.max().values
min = masked_scaled.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((masked_scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


/tmp/ipykernel_2842/4208043717.py:8: RuntimeWarning: invalid value encountered in cast
  img = ((masked_scaled.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)


In [16]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [17]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": masked_scaled.y[: -pad],
    "x": masked_scaled.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
masked_plus = masked_scaled.copy()
masked_plus = masked_plus.assign(result_bands)

masked_plus

<xarray.Dataset> Size: 31MB
Dimensions:      (y: 490, x: 262)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/35)
    nir          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    red          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    green        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    emad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    smad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    ...           ...
    homogeneity  (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    energy       (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    ASM          (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    correlation  (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    mean         (y, x) float32 514kB 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan
    entropy      (y, x) float32 514kB -1.443e-10 -1.443e-10 ... nan nan

In [32]:
# Re-apply the mask
masked_plus = masked_plus.where(all_masks)
masked_plus = masked_plus.drop_vars("count")
masked_da = masked_plus.to_dataarray()
masked_da = masked_da.squeeze()#.stack(dims=["y", "x"])#.transpose()
stacked_arrays_2d = masked_da.stack(new_dim=("y", "x")) 
reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')
stacked_arrays_2d.shape

# Replace any infinities with NaN
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("inf"))
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("-inf"))

# Replace any NaN values with 0
df = stacked_arrays_2d.squeeze().fillna(0).transpose().to_pandas()

# Remove the all-zero rows
zero_mask = (df == 0).all(axis=1)  # Creates a boolean Series
non_zero_df = df.loc[~zero_mask]  # Filters out all-zero rows

# Create a new array to hold the predictions
full_pred = pd.Series(np.nan, index=df.index)

reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')

### Model training

In [33]:
model = joblib.load("models/nm-15072025-test.model")

In [41]:
# reordered_data_array = reordered_data_array.drop_vars("count")
# reordered_data_array

In [36]:
# Predict the classes
predicted = model.predict(reordered_data_array)

In [37]:
# Reshape back to the original 2D array
reordered_data_array = predicted.reshape(len(masked_plus.y), len(masked_plus.x))

# Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    reordered_data_array, coords={"y": masked_plus.y, "x": masked_plus.x}, dims=["y", "x"]
)

In [38]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

# Check for NaN values
if np.isnan(predicted_da).any():
    print("NaN values found in the data")
    # Handle NaN values, e.g. by filling them
    predicted_da = predicted_da.fillna(0)  # Replace NaN with 0 or appropriate value

object


In [39]:
from matplotlib import colors

classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#ffffff"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

# predicted_da.plot.imshow(cmap=c_map, norm=norm, size=10)
predicted_da.odc.explore(cmap=c_map, tiles=basemaps.Esri.WorldImagery)

In [40]:
from matplotlib import colors

classes = [
    [1, "sediment", "#00000000"],
    [2, "sand", "#00000000"],
    [3, "rubble", "#00000000"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#00000000"],
    [6, "coral", "#00000000"],
    [7, "rock", "#00000000"],
    [8, "deeps", "#00000000"],
    [9, "mangrove", "#00000000"],
    [10, "land", "#00000000"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map_seagrass = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

predicted_da.odc.explore(cmap=c_map_seagrass, tiles=basemaps.Esri.WorldImagery)
# predicted_da.plot.imshow(cmap=c_map_seagrass, norm=norm, size=10)